<h1 style="color:green">Stroke Prediction</h1>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as scn

In [4]:
df = pd.read_csv("L://python//stroke.csv")
#df.head(10)
df.shape

(5110, 12)

In [5]:
df.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [6]:
df['stroke'].value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [7]:
df.dropna(inplace = True)

In [8]:
df.shape

(4909, 12)

In [9]:
df['stroke'].value_counts()

0    4700
1     209
Name: stroke, dtype: int64

In [10]:
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [11]:
gender = pd.get_dummies(df['gender'])
gender.drop('Other' , axis = 1 , inplace = True)
#gender

In [12]:
married = pd.get_dummies(df['ever_married'] , drop_first = True)
#married

In [13]:
resident = pd.get_dummies(df['Residence_type'],drop_first = True)
#resident

In [14]:
work = pd.get_dummies(df['work_type'])
work.drop('children' , axis = 1 , inplace = True)
#work

In [15]:
smoking = pd.get_dummies(df['smoking_status'] , drop_first = True)
#smoking

In [16]:
final = pd.concat([df , gender , married , resident , work , smoking] , axis = 1)

In [17]:
final.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,Male,Yes,Urban,Govt_job,Never_worked,Private,Self-employed,formerly smoked,never smoked,smokes
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,...,1,1,1,0,0,1,0,1,0,0
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,...,1,1,0,0,0,1,0,0,1,0
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,...,0,1,1,0,0,1,0,0,0,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,...,0,1,0,0,0,0,1,0,1,0
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,...,1,1,1,0,0,1,0,1,0,0


In [18]:
final.drop(['gender' , 'ever_married' , 'work_type' , 'Residence_type' , 'smoking_status'] , axis = 1 , inplace = True)

In [19]:
final.rename(columns= {'Yes':'married'} , inplace = True)
final.columns

Index(['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'stroke', 'Female', 'Male', 'married', 'Urban', 'Govt_job',
       'Never_worked', 'Private', 'Self-employed', 'formerly smoked',
       'never smoked', 'smokes'],
      dtype='object')

In [20]:
final.drop('id' , axis = 1 , inplace=True)

# start of KNN algorithm

In [21]:
import math
def distance(instance1 , instance2):
    length = len(instance1) - 1
    distance = 0
    for x in range(length):
        distance += abs(instance1[x]**2 - instance2[x]**2)
    return math.sqrt(distance)

In [22]:
def getneighbour(tr_set , test_ins , k):
    temp = tr_set.copy()
    length = len(temp)
    for i in range(length):
        dist = distance(temp[i] , test_ins)
        temp[i].append(dist)
    temp.sort(key=lambda x:x[-1])
    neighbours = []
    for j in range(k):
        neighbours.append((temp[j][0],temp[j][-2],temp[j][-1]))
    return neighbours        

In [23]:
def get_response(neighbours):
    vote = {}
    for ele in neighbours:
        if ele[-2] in vote:
            vote[ele[-2]] += 1
        else:
            vote[ele[-2]] = 1
    res = sorted(vote.items() , key=lambda x:x[0] , reverse = True)
    return res
        

In [24]:
def percent(res):
    total = 0
    for ele in res:
        total += ele[1]
    if int(res[0][0]) == 1:
        fav = res[0][1]
        per = (fav/total)*100
        return f'Chances of occurance of a stroke is {per} %'
    else:
        fav = res[0][1]
        per = (fav/total)*100
        return f'Chances of occurance of a stroke is {100 - per} %'

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
final['stroke_occurs'] = final['stroke']
X = final.drop('stroke' , axis = 1)
Y = final['stroke']

In [27]:
train_x , train_y , test_x , test_y = train_test_split(X , Y , test_size=0.01 , random_state = 1)

In [28]:
train_x

,age,hypertension,heart_disease,avg_glucose_level,bmi,Female,Male,married,Urban,Govt_job,Never_worked,Private,Self-employed,formerly smoked,never smoked,smokes,stroke_occurs
2345,46.0,0,0,85.81,20.2,1,0,1,0,0,0,1,0,1,0,0,0
4732,45.0,0,0,108.03,37.3,1,0,1,1,0,0,1,0,0,1,0,0
1104,62.0,0,0,204.57,34.4,0,1,1,0,1,0,0,0,0,0,0,0
963,30.0,0,0,73.69,17.3,1,0,0,0,0,0,1,0,0,1,0,0
3890,45.0,1,0,74.28,37.2,0,1,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,40.0,0,0,104.64,24.9,0,1,1,1,0,0,1,0,0,0,0,0
2899,55.0,0,0,68.79,27.0,0,1,1,1,0,0,1,0,0,1,0,0
962,33.0,0,0,77.42,26.1,0,1,0,0,0,0,1,0,0,0,0,0
4157,14.0,0,0,111.76,24.8,1,0,0,0,0,0,0,0,0,0,0,0


In [29]:
#train_y.head(10)
#train_y['stroke_occurs'].value_counts()
train = train_y[train_y['stroke_occurs']==1]
train

,age,hypertension,heart_disease,avg_glucose_level,bmi,Female,Male,married,Urban,Govt_job,Never_worked,Private,Self-employed,formerly smoked,never smoked,smokes,stroke_occurs


# Test Instance

 <pre style="color:blue">parameters = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'Female', 'Male', 'married', 'Urban', 'Govt_job',
       'Never_worked', 'Private', 'Self-employed', 'formerly smoked',
       'never smoked', 'smokes', 'stroke']</pre>

 <pre style="color:red">Last element of list<sub>test instance</sub> is as follows:
                                        case '1': positive chances of occurance
                                        case '0': negative chances of occurance</pre>

In [58]:
#test_ins = train_y.loc[71]
#print(list(test_ins))    
age = int(input("enter your age: "))
hyp = int(input("Hypertension 0/1: "))
h_dis = int(input("Heart Disease 0/1: "))
avg_glu = float(input("enter your avg glucose level: "))
bmi = float(input("enter your bmi: "))
fem = int(input("Female 0/1: "))
male = int(input("Male 0/1: "))
marr = int(input("married 0/1: "))
urb = int(input("urban 0/1: "))
govt = int(input("Govt job 0/1: "))
nev_wrk = int(input("Never worked 0/1: "))
pvt = int(input("private 0/1: "))
s_emp = int(input("self employed 0/1: "))
for_sm = int(input("Formarly smoked 0/1: "))
nev_sm = int(input("Never smoked 0/1: "))
sm = int(input("smoked 0/1: "))

test_ins = [age, hyp, h_dis, avg_glu, bmi, fem, male, marr, urb, govt, nev_wrk, pvt, s_emp, for_sm, nev_sm, sm, 0.0]

enter your age: 52
Hypertension 0/1: 1
Heart Disease 0/1: 0
enter your avg glucose level: 95
enter your bmi: 23
Female 0/1: 0
Male 0/1: 1
married 0/1: 1
urban 0/1: 1
Govt job 0/1: 0
Never worked 0/1: 0
private 0/1: 0
self employed 0/1: 1
Formarly smoked 0/1: 1
Never smoked 0/1: 0
smoked 0/1: 0


# Training set

In [59]:
train_set = []
for i in range(len(train_x)):
    train_set.append(list(train_x.iloc[i]))
print(train_set[0])

[46.0, 0.0, 0.0, 85.81, 20.2, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


### process

In [60]:
near_neigh = 100
neighbours = getneighbour(train_set , list(test_ins) , k = near_neigh)
neighbours

[(52.0, 0.0, 6.873107012116028),
 (51.0, 0.0, 14.3744947737303),
 (50.0, 0.0, 17.86204075686762),
 (51.0, 0.0, 18.097789920318995),
 (53.0, 0.0, 18.641912455539522),
 (50.0, 0.0, 19.192188515122517),
 (53.0, 0.0, 20.30117976867354),
 (51.0, 0.0, 20.5479658360627),
 (50.0, 0.0, 21.10998815726811),
 (52.0, 1.0, 21.92231967653058),
 (49.0, 0.0, 22.48831029668528),
 (52.0, 0.0, 22.51538140916111),
 (53.0, 0.0, 22.67047419001201),
 (52.0, 0.0, 22.68000220458539),
 (51.0, 0.0, 22.85368898011872),
 (53.0, 0.0, 23.016550566929002),
 (51.0, 0.0, 23.205764370086985),
 (55.0, 0.0, 24.023788210854654),
 (51.0, 0.0, 24.663991972103787),
 (55.0, 0.0, 24.976939364141476),
 (52.0, 0.0, 25.18267062882728),
 (52.0, 0.0, 25.606688579353627),
 (54.0, 0.0, 25.98591733997474),
 (51.0, 0.0, 26.469106520621366),
 (56.0, 0.0, 26.902302875404576),
 (54.0, 0.0, 27.325407590738703),
 (46.0, 0.0, 27.46221221970294),
 (57.0, 0.0, 27.85516110167022),
 (50.0, 0.0, 28.196615045072335),
 (47.0, 0.0, 28.289425586250452)

In [61]:
resp = get_response(neighbours)

In [62]:
print(resp)

[(1.0, 4), (0.0, 96)]


## patients Data

In [63]:
features = ['age (float)', 'hypertension (binary)', 'heart_disease (binary)', 'avg_glucose_level (float)',
       'bmi (float)', 'Female (binary)', 'Male (binary)', 'married (binary)', 'Urban (binary)', 'Govt_job (binary)',
       'Never_worked (binary)', 'Private (binary)', 'Self-employed (binary)', 'formerly smoked (binary)',
       'never smoked (binary)', 'smokes (binary)', 'Original_stroke_occurs (binary)']
for fea , val in zip(features , test_ins):
    print(fea , str(val).rjust(10))

age (float)         52
hypertension (binary)          1
heart_disease (binary)          0
avg_glucose_level (float)       95.0
bmi (float)       23.0
Female (binary)          0
Male (binary)          1
married (binary)          1
Urban (binary)          1
Govt_job (binary)          0
Never_worked (binary)          0
Private (binary)          0
Self-employed (binary)          1
formerly smoked (binary)          1
never smoked (binary)          0
smokes (binary)          0
Original_stroke_occurs (binary)        0.0


## predicted result

In [64]:
percentage = percent(resp)
percentage

'Chances of occurance of a stroke is 4.0 %'